In [ ]:
import argparse
import os
import sys
from functools import partial
from qiskit import IBMQ
import numpy as np
import networkx as nx
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.utils import QuantumInstance
from qiskit.opflow import AerPauliExpectation, PauliExpectation, StateFn, CircuitStateFn
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import Maxcut, SKModel
# from qiskit_optimization.translators import from_docplex_mp
from qiskit.utils import algorithm_globals
# from docplex.mp.model import Model


: 

In [2]:

parser = argparse.ArgumentParser()
parser.add_argument("--problem", type=str, default="maxcut")
parser.add_argument("-n", type=int, default=16)
parser.add_argument("-p", type=int, default=1)
parser.add_argument("-s", "--seed", type=int, default=0)
parser.add_argument("-b", "--backend", type=str, default="sv")
parser.add_argument("--cpu", default=False, action="store_true")
parser.add_argument("--no-aer", dest="aer", default=True, action="store_false")
parser.add_argument("--noise", type=str, default="ideal")
parser.add_argument("--p1", type=float, default=0.001)
parser.add_argument("--p2", type=float, default=0.005)
parser.add_argument("--beta-steps", type=int, default=50)
parser.add_argument("--gamma-steps", type=int, default=100)


_StoreAction(option_strings=['--gamma-steps'], dest='gamma_steps', nargs=None, const=None, default=100, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [3]:
def generate_landscape(args):
    print(args.__dict__)
    n = args.n
    p = args.p
    seed = args.seed
    backend_config = args.backend.lower()
    noise = args.noise.lower()

    if backend_config == "sv":
        method = "statevector"
    else:
        raise NotImplementedError(f"Backend {backend_config} not implemented yet")

    ## Add provider details
    backend = provider.get_backend('ibm_lagos')


    # --------------- define the problem ------------

    if args.problem == "maxcut":
        graph = nx.random_regular_graph(3, n, seed)
        problem = Maxcut(graph).to_quadratic_program()
    elif args.problem == "skmodel":
        problem = SKModel(n, seed).to_quadratic_program()
    else:
        raise NotImplementedError(f"{args.problem} problem is not implemented")

    H, offset = problem.to_ising()

    algorithm_globals.random_seed = seed

    # TODO: you might need to modify this to config IBM backend
    quantum_instance = QuantumInstance(
        backend=backend,
        seed_simulator=seed,
        seed_transpiler=seed,
        optimization_level = None,
    )

    algorithm = QAOA(
        SPSA(),
        reps=p,
        quantum_instance=quantum_instance,
        expectation=AerPauliExpectation() if args.aer else None,
    )
    algorithm._check_operator_ansatz(H)
    energy_evaluation, expectation = algorithm.get_energy_evaluation(
        H, return_expectation=True
    )

    # ------------- transpile ----------------
    def energy_evaluation_transpiled(parameters: np.ndarray) -> float:
        qc = quantum_instance.transpile(algorithm.ansatz.bind_parameters(parameters))[0]
        return qc

    energy_evaluation = energy_evaluation_transpiled
    
    

#     # ------------- prepare parameters and execute the circuit --------------
    data = []
    lst_circs = []
    lst_params = []
    beta_bound = np.pi / 4 / p
    gamma_bound = np.pi / 2 / p

    grid = np.array(
        np.meshgrid(
            *np.linspace([-beta_bound] * p, [beta_bound] * p, args.beta_steps, axis=1),
            *np.linspace([-gamma_bound] * p, [gamma_bound] * p, args.gamma_steps, axis=1),
            indexing="ij",
        )
    )
    
    
    for params in grid.transpose((*range(1, 2 * p + 1), 0)).reshape(-1, 2 * p):
        circ = energy_evaluation_transpiled(params) # ! execute the circuit
        circ.measure_all()
        tcirc=transpile(circ, backend=backend, seed_transpiler=11, optimization_level=3)      
        lst_circs.append(tcirc)
        lst_params.append(params)
        
    
    return (lst_circs,lst_params)



In [5]:
import pickle
from pathlib import Path




for p in [1]:
    for seed in [1]: # three instances
        cmd = f"-n 6 -p {p} --seed {seed} --noise depolar --p1 0.0001 --p2 0.008"
        args = parser.parse_args(list(cmd.split(' ')))
        lst_circs,lst_params = generate_landscape(args)
        path = "./IBM-M2/"
        name = "Problem-"+str(seed)+"P="+str(p)+"circ_param_lst.pkl"
        with open(path+name, 'wb') as f:
            pickle.dump([lst_circs,lst_params], f)
        
        

{'problem': 'maxcut', 'n': 6, 'p': 1, 'seed': 1, 'backend': 'sv', 'cpu': False, 'aer': True, 'noise': 'depolar', 'p1': 0.0001, 'p2': 0.008, 'beta_steps': 50, 'gamma_steps': 100}


ibmqfactory.load_account:WARNING:2023-02-14 01:49:40,243: Credentials are already in use. The existing account in the session will be replaced.


In [6]:
mynewlist=[]
for p in [1]:
    for seed in [1]: # three instances
        path = "./IBM-M2/"
        name = "Problem-"+str(seed)+"P="+str(p)+"circ_param_lst.pkl"
        with open(path+name, 'rb') as f:
            mynewlist = pickle.load(f)
        

In [7]:
circ_lst = []
for circ in mynewlist[0]:
    circ.remove_final_measurements()
    circ.measure_active()
    circ_lst.append(circ)

In [ ]:
## Add provider details

In [9]:
job_limit = backend.job_limit()
print(job_limit.maximum_jobs)

20


In [10]:
M1_Jobs =[]
for i in range(0,len(circ_lst),299):
    
    
    if i+299>len(circ_lst):
        job = backend.run(circ_lst[i:len(circ_lst)],shots=1024, memory=True)
        M1_Jobs.append(job)
        job.status()
    else:    
        job = backend.run(circ_lst[i:i+299],shots=1024, memory=True)
        job.status()
        M1_Jobs.append(job)

    

In [11]:
path = "./IBM-M2/"
name = "Problem-"+str(seed)+"P="+str(p)+"circ_param_lst.pkl"
with open(path+'results'+name, 'wb') as f:
    pickle.dump(M1_Jobs, f)

C:\Users\swamit.cs.wisc.edu\AppData\Local\Temp\ipykernel_27540\1061908471.py:4: DeprecationWarning: The `backends` provider attribute is deprecated. Please use `provider.backend` (singular) instead. You can continue to use `provider.backends()` to retrieve all backends.
  pickle.dump(M1_Jobs, f)


In [12]:
all_counts = []
for job in M1_Jobs:
    count_lst = job.result().get_counts()
    all_counts.append(count_lst) 
    

In [14]:
import itertools
list2d = all_counts
merged = list(itertools.chain.from_iterable(list2d))
path = "./IBM-M2/"
name = "Experimental_Data,Seed-"+str(seed)+"P="+str(p)+"circ_param_lst.pkl"
with open(path+name, 'wb') as f:
    pickle.dump([merged,mynewlist], f)